# Space Station 🚀
## SnowEx Hackweek 2022

## Motivation

- Can we build a pipeline to connect a SNOTEL stations observations to the closest relavent SnowEx data?
- How does a SNOTEL station represent surrounding measured snow from ASO's snow depth product?
- What are the main drivers for the surrounding heterogeneity (distance, slope, veg cover, aspect, elevation, etc.)?

## Process
![Diagram showing our process](path/to/diagram.png)

![Diagram showing raster to depth](./figures/raster-depth-diff.jpg)

## Locations

In [ ]:
# Show Explore tool here to browse around

### Map with All Locations
![Show ArcPro figures here](figures/ZoomedOut.png)

### Location close-up with Buffer
![Show ArcPro figures here](figures/ZoomedIn.png)

## Results and Visualizations

### Visualize depth difference between snow depth raster and SNOTEL snow dpeth

In [ ]:
# Show Point map with difference values 

### Some rudimentary stats 

In [ ]:
# Show histograms and other stats products here

## Next Steps

